# POS Tagging using Maximum Entropy Markov Models
A first-order maximum entropy Markov model will be implemented for the purpose of part-of-speech tagging. In the process, we will implement Viterbi decoding and featurize the text for the MEMM.

## Setup

In [1]:
import sys 
import math
import numpy as np
from collections import Counter
from sklearn import linear_model
from scipy import sparse
import pickle

# Dictionary to store indices for each feature
feature_vocab = {}

# Dictionary to store the indices of each POS tag
label_vocab = {}

def load_data(filename):
    """
        load data from filename and return a list of lists
        all_toks = [toks1, toks2, ...] where toks1 is
                a sequence of words (sentence)

        all_labs = [labs1, labs2, ...] where labs1 is a sequence of
                labels for the corresponding sentence
    """
    file = open(filename)
    all_toks = []
    all_labs = []
    toks = []
    labs = []
    for line in file:
        if "This data is licensed from" in line:
            continue
        cols = line.rstrip().split("\t")
        if len(cols) < 2:
            all_toks.append(toks)
            all_labs.append(labs)
            toks = []
            labs = []
            continue
        toks.append(cols[0])
        labs.append(cols[1])

    if len(toks) > 0:
        all_toks.append(toks)
        all_labs.append(labs)

    return all_toks, all_labs

def decode(Y_pred):
    if use_greedy:
        return greedy_decode(Y_pred)
    else:
        return viterbi_decode(Y_pred)

In the next cell there are some constants that will be used for the MEMM.

In [2]:
# Control variables, use for debugging and testing
verbose = True
use_greedy = False

# Minimum number of observations for a feature to be included in model
MINIMUM_FEAT_COUNT = 2 
# L2 regularization strength; range is (0,infinity), with stronger regularization -> 0 (in this package)
L2_REGULARIZATION_STRENGTH = 0.9 

TRAINING_FILE = 'wsj.pos.train'
TEST_FILE = 'wsj.pos.test'

## 1. Decoding
Following are functions that perform decoding: the trivial greedy approach and a Viterbi decoding implementation

In [3]:
def greedy_decode(Y_pred):
    """ 
        greedy decoding to get the sequence of label predictions
    """
    cur = label_vocab["START"]
    preds = []
    for i in range(len(Y_pred)):
        pred = np.argmax(Y_pred[i, cur])
        preds.append(pred)
        cur = pred
    return preds

In [4]:
def viterbi_decode(Y_pred):
    """
        :return
        list of POS tag indices, defined in label_vocab,
        in order of sequence

        :param
        Y_pred: Tensor of shape N * M * L, where
        N is the number of words in the current sequence,
        L is the number of POS tag labels
        M = L + 1, where M is the number of possible previous labels
        which includes L + "START"

        M_{x,y,z} is the probability of a tag (label_vocab[current_tag] = z)
        given its current position x in the sequence and
        its previous tag (label_vocab[previous_tag] = y)

        Consider the sentence - "I have a dog". Here, N = 4.
        Assume that there are only 3 possible tags: {PRP, VBD, NN}
        M_{0, 3, 2} would give you the probability of "I" being a "NN" if
        it is preceded by "START". "START" is the last index of all lablels,
        and in our example denoted by 3.
    """
    (N, M, L) = Y_pred.shape

    # list of POS tag indices to be returned
    path = []

    """
        step 1. construct a Viterbi matrix to store the
        intermediate joint probabilities
        step 2. construct a matrix to store the backpointers
        to retrieve the path
        step 3. decode and return the path from the Viterbi matrix
    """
    viterbi_matrix = np.zeros(shape=(N, L)) 
    bps = np.zeros(shape=(N, L), dtype=np.int32)
    for word in range(N):
        if word == 0: # START
            bps[word] = np.array([Y_pred[0].shape[0] - 1] * L)
            viterbi_matrix[word] = np.log(Y_pred[word, Y_pred[0].shape[0] - 1])
        else:
            for tag in range(L):
                viterbi = [viterbi_matrix[word-1, prev_tag] + np.log(Y_pred[word, prev_tag, tag]) for prev_tag in range(L)]
                bps[word, tag] = np.argmax(viterbi)
                viterbi_matrix[word, tag] = np.max(viterbi)
    
    # start off with node with highest value
    node = np.argmax(viterbi_matrix[N-1])
    path.append(node)
    
    # tracing the backpointers to form path
    i = N - 1
    while i > 0:
        path.append(bps[i, node])
        node = bps[i, node]
        i -= 1

    return path[::-1]

## 2. Features
The `get_features` function adds features for the maximum entropy Markov model to use. The features are explained in detail in the README.

In [5]:
def get_features(index, sequence, tag_index_1):
    """
        :params
        index: the index of the current word in the sequence to featurize

        sequence: the sequence of words for the entire sentence

        tag_index_1: gives you the POS
        tag for the previous word.

        :return (feature dictionary)
        features are in the form of {feature_name: feature_value}
    """
    features = {}

    features["UNIGRAM_%s" % sequence[index].lower()] = 1
    features["PREVIOUS_TAG_%s" % tag_index_1] = 1

    # Feature Class 1: Miscellaneous --------------------------------------------------
    
    features["WORD_{}_PREVTAG_{}".format(sequence[index].lower(), tag_index_1)] = 1
    
    if sequence[index].strip()[0].isupper():
        if all(c.isupper() for c in sequence[index].strip()):
            features["ALL_CAPITAL"] = 1
        else:
            features["CAPITALIZED"] = 1
   
    if any(c.isdigit() for c in sequence[index]):
        features["CONTAINS_NUMBER"] = 1
    
    if '-' in sequence[index]:
        features['HYPHENATED'] = 1
    
    # Feature Class 2: Bigrams --------------------------------------------------

    if index != 0: # past word bigram
        features["BIGRAM_{0}_{1}".format(sequence[index].lower(), sequence[index-1].lower())] = 1
    else:
        features["FIRST_WORD_IN_SEQUENCE"] = 1

    if index != len(sequence) - 1: # future word bigram
        features["BIGRAM_{0}_{1}".format(sequence[index].lower(), sequence[index+1].lower())] = 1
    else:
        features["LAST_WORD_IN_SEQUENCE"] = 1
        
    # Feature Class 3: Trigrams --------------------------------------------------
        
    if index >= 1 and index < len(sequence) - 1: # trigram of past and future words
        w1 = sequence[index].lower()
        w2 = sequence[index-1].lower()
        w3 = sequence[index+1].lower()
        features["TRIGRAM_{0}_{1}_{2}".format(w2, w1, w3)] = 1
        
    # Feature Class 4: Prefixes --------------------------------------------------
    
    chk = sequence[index].strip()
    
    features["PREFIX_{0}".format(sequence[index]).lower()[:3]] = 1
    
    if chk.startswith("anti"):
        features['STARTS_anti'] = 1
    
    if chk.startswith("pre"):
        features['STARTS_pre'] = 1
    
    if chk.startswith("dis"):
        features['STARTS_dis'] = 1
    
    if chk.startswith("mis"):
        features['STARTS_mis'] = 1
    
    if chk.startswith("inter"):
        features['STARTS_inter'] = 1
    
    if chk.startswith("un"):
        features['STARTS_un'] = 1
        
    if chk.startswith("non"):
        features['STARTS_non'] = 1
    
    if chk.startswith("over") or chk.startswith("under"):
        features['STARTS_over_under'] = 1
    
    if chk.startswith("in") or chk.startswith("im"):
        features['STARTS_in_im'] = 1
    
    if chk.startswith("en") or chk.startswith("em"):
        features['STARTS_en_em'] = 1
   
    # Feature Class 5: Suffixes --------------------------------------------------
    
    features["SUFFIX_{0}".format(sequence[index]).lower()[-2:]] = 1
    
    if chk.endswith("ly"):
        features['ENDS_ly'] = 1
    
    if chk.endswith("ing"):
        features['ENDS_ing'] = 1
    
    if chk.endswith("ness"):
        features['ENDS_ness'] = 1
        
    if chk.endswith("ity"):
        features['ENDS_ity'] = 1
    
    if chk.endswith("ed"):
        features['ENDS_ed'] = 1
    
    if chk.endswith("able"):
        features['ENDS_able'] = 1
    
    if chk.endswith("s"):
        features['ENDS_s'] = 1
        
    if chk.endswith("ion"):
        features['ENDS_ion'] = 1
        
    if chk.endswith("al"):
        features['ENDS_al'] = 1
    
    if chk.endswith("ive"):
        features['ENDS_ive'] = 1
    
    if chk.endswith("er"):
        features['ENDS_er'] = 1
    
    if chk.endswith("est"):
        features['ENDS_est'] = 1
    
    if chk.endswith("day"):
        features['ENDS_day'] = 1
        
    # Feature Class 6: Specials --------------------------------------------------
    
    if sequence[index].strip()[0].isupper():
        copy_back = index - 1
        copy_front = index + 1
        while copy_back >= 1 and (index-copy_back) <= 3:
            chk = sequence[copy_back].strip().lower()
            if "co." in chk or "inc." in chk or "corp" in chk or "company" in chk or "llc" in chk or "ltd" in chk:
                features['COMPANY'] = 1
                break
            copy_back -= 1
        while copy_front < len(sequence) and (copy_front-index) <= 3:
            chk = sequence[copy_front].strip().lower()
            if "co." in chk or "inc." in chk or "corp" in chk or "company" in chk or "llc" in chk or "ltd" in chk:
                features['COMPANY'] = 1
                break
            copy_front += 1
        
    if index > 1:
        chk = sequence[index - 1].strip().lower()
        if chk == "a" or chk == "an" or chk == "the":
            features['PREV_DETERMINER'] = 1
            
    return features

With both Viterbi decoding implemented and features added, we can now build and train our MEMM!

In [6]:
def train(filename):
    """
        train a model to generate Y_pred
    """
    all_toks, all_labs = load_data(filename)
    vocab = {}

    # X_verbose is a list of feature objects for the entire train dataset.
    # Each feature object is a dictionary defined by get_features and corresponds to a word.
    # Y_verbose is a list of labels for all words in the entire train dataset
    X_verbose = []
    Y_verbose = []

    feature_counts=Counter()

    for i in range(len(all_toks)):
        toks = all_toks[i]
        labs = all_labs[i]
        for j in range(len(toks)):
            prev_lab = labs[j - 1] if j > 0 else "START"
            feats = get_features(j, toks, prev_lab)
            X_verbose.append(feats)
            Y_verbose.append(labs[j])

            for feat in feats:
                feature_counts[feat]+=1

    # construct label_vocab (dictionary) and feature_vocab (dictionary)
    # label_vocab[pos_tag_label] = index_for_the_pos_tag
    # feature_vocab[feature_name] = index_for_the_feature
    feature_id = 1
    label_id = 0

    # create unique integer ids for each label and each feature above the minimum count threshold
    for i in range(len(X_verbose)):
        feats = X_verbose[i]
        true_label = Y_verbose[i]

        for feat in feats:
            if feature_counts[feat] >= MINIMUM_FEAT_COUNT:
                if feat not in feature_vocab:
                    feature_vocab[feat] = feature_id
                    feature_id += 1
        if true_label not in label_vocab:
            label_vocab[true_label] = label_id
            label_id += 1

    # START has last id
    label_vocab["START"] = label_id

    # create train input and output to train the logistic regression model
    # create sparse input matrix

    # X is documents x features empty sparse matrix
    X = sparse.lil_matrix((len(X_verbose), feature_id))
    Y = []

    print("Number of features: %s" %len(feature_vocab))
    for i in range(len(X_verbose)):
        feats = X_verbose[i]
        true_label = Y_verbose[i]
        for feat in feats:
            if feat in feature_vocab:
                X[i, feature_vocab[feat]] = feats[feat]
        Y.append(label_vocab[true_label])

    # fit model
    log_reg = linear_model.LogisticRegression(C=L2_REGULARIZATION_STRENGTH, penalty='l2')
    log_reg.fit(sparse.coo_matrix(X), Y)

    return log_reg


def test_dev(filename, log_reg):
    """
        predict labels using the trained model
        and evaluate the performance of model
    """
    all_toks, all_labs = load_data(filename)

    # possible output labels = all except START
    L = len(label_vocab) - 1

    correct = 0.
    total = 0.

    num_features = len(feature_vocab) + 1

    # for each sequence (sentence) in the test dataset
    for i in range(len(all_toks)):

        toks = all_toks[i]
        labs = all_labs[i]

        if len(toks) == 0:
            continue

        N = len(toks)

        X_test = []
        # N x prev_tag x cur_tag
        Y_pred = np.zeros((N, L + 1, L))

        # vector of true labels
        Y_test = []

        # for each token (word) in the sentence
        for j in range(len(toks)):

            true_label = labs[j]
            Y_test.append(true_label)

            # for each preceding tag of the word
            for possible_previous_tag in label_vocab:
                X = sparse.lil_matrix((1, num_features))

                feats = get_features(j, toks, possible_previous_tag)
                valid_feats = {}
                for feat in feats:
                    if feat in feature_vocab:
                        X[0, feature_vocab[feat]] = feats[feat]

                # update Y_pred with the probabilities of all current tags
                # given the current word, previous tag and other data/feature
                prob = log_reg.predict_proba(X)
                Y_pred[j, label_vocab[possible_previous_tag]] = prob

        # decode to get the predictions
        predictions = decode(Y_pred)

        # evaluate the performance of the model by checking predictions
        # against true labels
        for k in range(len(predictions)):
            if Y_test[k] in label_vocab and predictions[k] == label_vocab[Y_test[k]]:
                correct += 1
            total += 1

        print("Development Accuracy: %.3f (%s/%s)." % (correct / total, correct, total), end="\r")

In [7]:
print("Train Model")
log_reg = train(TRAINING_FILE)
print("Test Model")
test_dev(TEST_FILE, log_reg)

Train Model
Number of features: 291877
Test Model
